In [26]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout

def load_data(filename, skiprows = 1):
    '''   
    Function loads data stored in the file filename and returns it as a numpy ndarray.\n",

    Inputs:
    filename: given as a string.
    Outputs:\n",
    Data contained in the file, returned as a numpy ndarray\n",
    '''
    return np.loadtxt(filename, skiprows=skiprows, delimiter=',')

Using TensorFlow backend.


In [166]:
from keras import backend as k_back


# define the AUC metric being used in the competition
# this function can be used in Keras' Sequential Model compilation only
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    k_back.get_session().run(tf.local_variables_initializer())
    return auc

In [204]:
train = load_data("data/train_2008.csv")
test = load_data("data/test_2008.csv")


In [18]:
y_train_new = train[:,train.shape[1]-1:]

In [110]:
def refineInput(X, y_train_new, section, total):
    X_traincopy = X.copy()
    y_traincopy = y_train_new.copy()
    X_train = X_traincopy[:,3:]
    num = X_traincopy.shape[0]/total
    X_test = X_traincopy[int(num*section):int(num*(section+1)),:]
    X_train = X_traincopy[:int(num*section),:]
    X_train = np.concatenate((X_train, X_traincopy[int(num*(section+1)):,:]))
    
    y_test = y_traincopy[int(num*section):int(num*(section+1)),:]
    y_train = y_traincopy[:int(num*section),:]
    y_train = np.concatenate((y_train, y_traincopy[int(num*(section+1)):,:]))
    
    return X_train, X_test, y_train, y_test

In [108]:
X_train[10:12,:]

array([[ 1.8405000e+04,  1.1000000e+01,  2.0080000e+03,  1.0000000e+00,
         2.0100000e+02,  0.0000000e+00,  2.0000000e+00,  1.0000000e+00,
         1.0000000e+00, -1.0000000e+00,  1.0000000e+00, -2.0000000e+00,
        -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,  4.3738474e+07,
         1.0000000e+00,  5.0000000e+00,  4.0000000e+00,  5.0000000e+00,
         2.0000000e+00,  1.0000000e+00,  3.0000000e+00,  8.3001000e+04,
         2.0000000e+00, -1.0000000e+00, -1.0000000e+00, -1.0000000e+00,
        -1.0000000e+00,  2.0000000e+00,  3.3000000e+01,  1.7000000e+01,
         1.9500000e+04,  1.1500000e+02,  4.0000000e+00,  1.0000000e+00,
         0.0000000e+00,  2.0000000e+00,  0.0000000e+00,  1.0000000e+00,
         4.0000000e+00,  1.8000000e+01,  0.0000000e+00,  6.0000000e+00,
        -1.0000000e+00,  2.0000000e+00,  2.0000000e+00,  2.0000000e+00,
         3.9000000e+01,  2.0000000e+00, -1.0000000e+00,  9.0000000e+00,
         1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.00000

In [ ]:
model = Sequential([
    #Flatten(input_shape=(28,28,)),  # Use np.reshape instead of this in hw\n",
    Dense(1050, input_shape=(382,)), 
    Activation('relu'), 
    Dropout(0.3),        
    Dense(1),
    Activation('softmax')
    ])
model.compile(loss='squared_hinge',optimizer='adam', metrics=['accuracy'], verbose=1)
model.build()
    
model.summary()
fit = model.fit(X_train, y_train_new, batch_size=32, epochs=3, verbose = 1)
## Printing the accuracy of our model, according to the loss function specified in model.compile above\n",
score = model.evaluate(X_train, y_train_new)
print(score)

predict = model.predict(X_train)




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 1050)              402150    
_________________________________________________________________
activation_22 (Activation)   (None, 1050)              0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 1050)              0         
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 1051      
_________________________________________________________________
activation_23 (Activation)   (None, 1)                 0         
Total params: 403,201
Trainable params: 403,201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
64667/64667 [==============================] - 25s 384us/step - loss: 0.7446 - acc: 0.2554
Epoch 2/10
64667/64667 [==========

In [141]:
X_train = train[:,:train.shape[1]-1]
#X_train = X_train[:,3:]
score = model.evaluate(X_train, y_train_new)
print(score)

64667/64667 [==============================] - 6s 88us/step
[0.7446147184799109, 0.25538528152008916]


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [171]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

N = 7
for i in range(7):
    y_train_new = train[:,train.shape[1]-1:]
    X_train = train[:,:train.shape[1]-1]
    X_train, x_test, y_train, y_test = refineInput(X_train, y_train_new, i, 7)
    # Fit regression model
    regr_1 = DecisionTreeRegressor(max_depth=7)
    regr_2 = DecisionTreeRegressor(max_depth=10)
    regr_1.fit(X_train, y_train)
    regr_2.fit(X_train, y_train)

    # Predict
    y_1 = regr_1.predict(x_test)
    y_2 = regr_2.predict(x_test)
    
    y_t_1 = regr_1.predict(X_train)
    y_t_2 = regr_2.predict(X_train)
    
    
    print("test", roc_auc_score(y_test, y_1))
    print(roc_auc_score(y_test, y_2)  )
    
    print("train", roc_auc_score(y_train ,y_t_1 ))
    print(roc_auc_score(y_train,y_t_2)  )


(55429, 382) (55429, 1)
test 0.24569092497610678
0.2724278963027511
train 0.22834767175919446
0.18382215233471133
(55429, 382) (55429, 1)
test 0.24093237515124077
0.2681201444441733
train 0.23018821503929998
0.19051035161426588
(55429, 382) (55429, 1)
test 0.25700321766530987
0.2778615252214859
train 0.22652252366797798
0.18272851154799685
(55429, 382) (55429, 1)
test 0.245460970566455
0.2685809975186084
train 0.2296259653046281
0.18563403573404924
(55429, 382) (55429, 1)
test 0.25148736244195025
0.2596522686880114
train 0.2281081888271741
0.18682707332699167
(55429, 382) (55429, 1)
test 0.2436451850078395
0.26706897174306
train 0.22927443062407538
0.1870351438697503
(55428, 382) (55428, 1)


KeyboardInterrupt: 

In [176]:
from sklearn.ensemble import RandomForestRegressor

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


N = 7
res = []
for depth in range(1, 10):
    row = []
    for nodes in range(2, 10):
        avg_test = 0
        avg_train = 0
        for i in range(7):
            y_train_new = train[:,train.shape[1]-1:]
            X_train = train[:,:train.shape[1]-1]
            X_train, x_test, y_train, y_test = refineInput(X_train, y_train_new, i, 7)
            # Fit regression model
            y_train = np.ravel(y_train)
            regr_1 = RandomForestRegressor(max_depth=depth, max_leaf_nodes = nodes)
            regr_1.fit(X_train, y_train)
            #regr_2.fit(X_train, y_train)

            # Predict
            y_1 = regr_1.predict(x_test)
            #y_2 = regr_2.predict(x_test)

            y_t_1 = regr_1.predict(X_train)
            #y_t_2 = regr_2.predict(X_train)

            avg_test += roc_auc_score(y_test, y_1)
            avg_train += roc_auc_score(y_train ,y_t_1 )
            #print("test", roc_auc_score(y_test, y_1))
            #print(roc_auc_score(y_test, y_2)  )

            #print("train", roc_auc_score(y_train ,y_t_1 ))
            #print(roc_auc_score(y_train,y_t_2)  )
        avg_test /= 7
        avg_train /= 7
        row.append(avg_test)
        print(avg_test)
    res.append(row)
    


(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55428, 382) (55428, 1)
0.6495285165738481
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55428, 382) (55428, 1)
0.67265743304632
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55428, 382) (55428, 1)
0.7105775482162505
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55428, 382) (55428, 1)
0.7111891221547529
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55428, 382) (55428, 1)
0.7106737243649933
(55429, 382) (55429, 1)
(55429, 382) (55429, 1)
(55429, 382) (55429

KeyboardInterrupt: 

In [69]:
from sklearn.metrics import mean_squared_error


0.054374085090559646
0.13803736922642404


In [206]:
X_test


array([], shape=(16000, 0), dtype=float64)

In [155]:
X_test = test[:,3:]
y_train_new = train[:,train.shape[1]-1:]
X_train = train[:,:train.shape[1]-1]
X_train = X_train[:,3:]
#regr_2 = DecisionTreeRegressor(max_depth=10)
#regr_2.fit(X_train, y_train_new)
#y_2 = regr_2.predict(X_test)

In [219]:
test = load_data("data/test_2008.csv")

from sklearn.preprocessing import MinMaxScaler
X_test = test[:,3:]
predictions = xgb.predict(X_test)
predictions = predictions.reshape(16000, 1)

mmscaler = MinMaxScaler()
new_pred = mmscaler.fit_transform(predictions)
ids = test[:,:1]
ids = ids.astype(int)
new_pred = new_pred.astype(float)
test = np.concatenate((ids, predictions), axis=1)

In [226]:

predictions = predictions.astype(float)
test = np.concatenate((ids, predictions), axis=1)

In [227]:
np.savetxt('submission.csv', test, delimiter=',', fmt='%1.10f', header = 'id,target')

In [225]:
print(predictions)
print(new_pred)
for i in predictions:
    if i[0] <0:
        i[0] = 0
    elif i[0]>1:
        i[0] = 1
        
print(min(predictions))

[[0.27371597]
 [0.11366704]
 [0.1329549 ]
 ...
 [0.24748915]
 [0.02386603]
 [0.2097044 ]]
[[0.29579499]
 [0.15811621]
 [0.17470819]
 ...
 [0.27323392]
 [0.08086675]
 [0.24073036]]
[0.]


In [201]:
import xgboost

from sklearn import cross_validation, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

y_train_new = train[:,train.shape[1]-1:]
X_train = train[:,:train.shape[1]-1]
    
X_test = test[:,3:]
y_train_new = train[:,train.shape[1]-1:]
X_train = train[:,:train.shape[1]-1]
X_train = X_train[:,3:]

#X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_train, y_train_new,test_size=0.2)

xgb.fit(X_train, y_train_new)


#print(roc_auc_score(B, y_test))




ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [203]:
X_test

array([], shape=(16000, 0), dtype=float64)

In [153]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor()

y_train = np.ravel(y_train)
nn.fit(X_train, y_train)
nn.score(X_train, y_train)

-23412360736.750458

In [152]:
y_train.shape

(51733, 1)

In [182]:
from sklearn import neighbors

y_train_new = train[:,train.shape[1]-1:]
X_train = train[:,:train.shape[1]-1]
    
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_train, y_train_new,test_size=0.2)
n_neighbors = 10000
knn = neighbors.KNeighborsRegressor(n_neighbors)
y_ = knn.fit(X_train, y_train).predict(X_test)

for i in range(10):
    print(y_[i], y_test[i])

print(roc_auc_score(y_test,y_)  )
    
    
    

[0.246] [0.]
[0.2556] [0.]
[0.2614] [0.]
[0.2691] [0.]
[0.2423] [0.]
[0.2643] [0.]
[0.2394] [0.]
[0.256] [0.]
[0.25] [0.]
[0.2477] [1.]
0.5141380755005418


In [183]:
from sklearn.svm import SVR

y_train_new = train[:,train.shape[1]-1:]
X_train = train[:,:train.shape[1]-1]
    
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_train, y_train_new,test_size=0.2)


clf = SVR(kernel='poly')
clf.fit(X_train, y_train) 

/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [ ]:
clf.predict(X_test)


In [228]:
xgprediction = predictions.copy()


In [237]:
xgprediction

array([[0.27371597],
       [0.11366704],
       [0.1329549 ],
       ...,
       [0.24748915],
       [0.02386603],
       [0.2097044 ]])

In [229]:
import csv
nn = []
with open('nn.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        nn.append(row[1])

In [233]:
nn_prediction = np.array(nn)

In [236]:
nn_prediction = nn_prediction.astype(float)


array([0.30951509, 0.13851742, 0.14212674, ..., 0.22869109, 0.12686066,
       0.30474904])

In [238]:
nn_prediction = np.reshape(nn_prediction, (-1, 1))

In [239]:
nn_prediction

array([[0.30951509],
       [0.13851742],
       [0.14212674],
       ...,
       [0.22869109],
       [0.12686066],
       [0.30474904]])

In [242]:
merger = (nn_prediction + xgprediction)/2

In [243]:
test = np.concatenate((ids, merger), axis=1)
np.savetxt('submission.csv', test, delimiter=',', fmt='%1.10f', header = 'id,target')